<a href="https://colab.research.google.com/github/yumed15/colabs-playground/blob/main/pinecone_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello, Pinecone!

This notebook will walk through the steps to get a simple Pinecone index up and running. 


### Prerequisites

Install dependencies.

In [1]:
!pip install -qU pinecone-client pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.6 MB/s eta 0:00:00


Set up Pinecone. Get your Pinecone API key [here](https://www.pinecone.io/start).

In [3]:
import pinecone
import os

# Load Pinecone API key
api_key = ''
pinecone.init(
    api_key=api_key,
    environment="asia-southeast1-gcp"  # find next to API key in console
)

## Pinecone quickstart

With Pinecone you can create a vector index where you can store and search through your vectors. 

In [4]:
# Giving our index a name
index_name = "hello-pinecone"
# Delete the index, if an index of the same name already exists
if index_name in pinecone.list_indexes():
    pinecone.delete_index(index_name)

Creating a Pinecone Index.

In [5]:
dimensions = 3
pinecone.create_index(name=index_name, dimension=dimensions, metric="cosine")

In [6]:
index = pinecone.Index(index_name=index_name)

We have the index ready. Now we will create some simple vectors that will serve as our examples.

In [7]:
import pandas as pd

df = pd.DataFrame(
    data={
        "id": ["A", "B"], 
        "vector": [[1., 1., 1.], [1., 2., 3.]]
    })
df

,id,vector
0,A,"[1.0, 1.0, 1.0]"
1,B,"[1.0, 2.0, 3.0]"


We perform upsert operations in our index. This call will insert a new vector in the index or update the vector if the id was already present.

In [8]:
index.upsert(vectors=zip(df.id, df.vector))  # insert vectors

{'upserted_count': 2}

In [9]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2}},
 'total_vector_count': 2}

In [10]:
index.query(
    vector=[2., 2., 2.], 
    top_k=5, 
    include_values=True) # returns top_k matches

{'matches': [{'id': 'A', 'score': 1.0, 'values': [1.0, 1.0, 1.0]},
             {'id': 'B', 'score': 0.925820112, 'values': [1.0, 2.0, 3.0]}],
 'namespace': ''}

## Delete the Index
Delete the index once you are sure that you do not want to use it anymore. Once the index is deleted, you cannot use it again.

In [ ]:
pinecone.delete_index(index_name)